In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
# Address：http://d2l-data.s3-accelerate.amazonaws.com/fra-eng.zip
import os
import torch
import sys
file_path = '/content/drive/MyDrive/Colab Notebooks/MT/dltools.py'

with open(file_path, 'r') as file:
    exec(file.read())

# Dataset loading



In [ ]:
"""Dataset loading"""

with open('/content/drive/MyDrive/Colab Notebooks/MT/Dataset/fra.txt', 'r',encoding='utf-8') as f:
    raw_text = f.read()

In [20]:
def preprocess_nmt(text):
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '


    # Replacing uppercase letters with lowercase letters
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()

    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


In [27]:
# Tokenization
def tokenize_nmt(text, num_examples=None):
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

source, target = tokenize_nmt(text)

In [ ]:
def show_list_len_pair_hist(legend, xlabel, ylabel, xlist, ylist):
    dltools.set_figsize()
    _, _, patches = dltools.plt.hist(
        [[len(l) for l in xlist], [len(l) for l in ylist]])
    dltools.plt.xlabel(xlabel)
    dltools.plt.ylabel(ylabel)
    for patch in patches[1].patches:
        patch.set_hatch('/')
    dltools.plt.legend(legend)

show_list_len_pair_hist(['source', 'target'], '# tokens per sequence','count', source, target);

In [36]:
# Build word list
src_vocab = dltools.Vocab(source, min_freq=2,reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab) # 10012

10012

# Load Dataset
truncation or padding 0

In [ ]:
def truncate_pad(line, num_steps, padding_token):
    if len(line) > num_steps:
        return line[:num_steps]  # truncate
    return line + [padding_token] * (num_steps - len(line))  # padding

truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

In [31]:
def build_array_nmt(lines, vocab, num_steps):
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

In [30]:
def load_data_nmt(batch_size, num_steps,num_examples=600):
    text = preprocess_nmt(raw_text)
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = dltools.Vocab(source, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = dltools.Vocab(target, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = dltools.load_array(data_arrays, batch_size)
    return data_iter, src_vocab, tgt_vocab

In [ ]:
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)
for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X.type(torch.int32))
    print('Valid length of X:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('Valid length of Y:', Y_valid_len)
    break